<a href="https://colab.research.google.com/github/Vkang10/Customer-Lifetime-Value-Prediction/blob/main/Creating_dataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Creating training, testing & holdout datasets

In [ ]:
# Authenticate using Google OAuth 2.0
# This will provide the credentials necessary to use Google APIs within this notebook (e.g. BigQuery API)
from google.colab import auth
auth.authenticate_user( )
print('Authenticated')

Authenticated


In [ ]:
# Query the Analytics 360 sample in BigQuery and store results as a Pandas dataframe
import pandas as pd
import numpy as np
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage

# Change the project to the ID of your personal GCP project
# You can find this ID by going to BQ, clicking on your project name at the top of the UI and finding the ID listed for your project name in the pop-up
project = 'project-501-9'

credentials, project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

bqclient = bigquery.Client(credentials=credentials, project=project,)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

# The query below finds the first transaction of each customer before 05/01/2017
# It summarizes some information from the session where that transaction occurred (e.g. revenue, time on site, page views, etc.)
# It then summarizes some information from subsequent sessions that happen within 7 days of the first transaction session (e.g. additional visits, additional transactions)
# It then sums any subsequent revenue received from that customer over the next 8-90 days, which will be the target for our model

sql = """ 
WITH first_purchases AS (
# Find the start time for the session when the customer made their first purchase
SELECT 
    fullVisitorId, # Consider this the customer ID
    MIN(PARSE_TIMESTAMP('%s', CAST(visitStartTime AS string))) AS firstPurchaseSessionTime 
FROM 
    `bigquery-public-data.google_analytics_sample.ga_sessions_*` ga
WHERE
    _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
    AND geoNetwork.country = 'United States' # Only looking at US acquisitions so we can assume everything is in USD
GROUP BY
    fullVisitorId
), 

first_purchases_sessions AS (
# Get session-level data from the session when the customer made their first purchase
# You'll see later that we have to query session-level data and hit-level data separately
SELECT 
    fp.fullVisitorId,
    ROUND(SUM(totals.transactionRevenue/1000000),2) AS revenue, 
    MAX(visitNumber) AS visitNumber,
    SUM(totals.hits) AS hits,
    SUM(totals.pageviews) AS pageviews,
    COALESCE(SUM(totals.timeOnSite),0) AS timeOnSite,
    MAX(CASE WHEN channelGrouping = 'Referral' THEN 1 ELSE 0 END) AS referralChannelGrouping,
    MAX(CASE WHEN channelGrouping = 'Organic Search' THEN 1 ELSE 0 END) AS organicSearchChannelGrouping,
    MAX(CASE WHEN channelGrouping = 'Direct' THEN 1 ELSE 0 END) AS directChannelGrouping,
    MAX(CASE WHEN channelGrouping = 'Paid Search' THEN 1 ELSE 0 END) AS paidSearchChannelGrouping,
    MAX(CASE WHEN device.browser = 'Chrome' THEN 1 ELSE 0 END) AS chromeBrowser,
    MAX(CASE WHEN device.browser = 'Safari' THEN 1 ELSE 0 END) AS safariBrowser,
    MAX(CASE WHEN device.operatingSystem = 'Macintosh' THEN 1 ELSE 0 END) AS macintoshOS,
    MAX(CASE WHEN device.operatingSystem = 'Windows' THEN 1 ELSE 0 END) AS windowsOS,
    MAX(CASE WHEN device.operatingSystem = 'Chrome OS' THEN 1 ELSE 0 END) AS chromeOS,
    MAX(CASE WHEN device.operatingSystem = 'Linux' THEN 1 ELSE 0 END) AS LinuxOS,
    MAX(CASE WHEN geoNetwork.region = 'California' THEN 1 ELSE 0 END) AS california, # dummy variable indicating if the customer was in California
    MAX(CASE WHEN geoNetwork.region = 'New York' THEN 1 ELSE 0 END) AS newYork, # dummy variable indicating if the customer was in New York
    MAX(CASE WHEN geoNetwork.metro = 'San Francisco-Oakland-San Jose CA' THEN 1 ELSE 0 END) AS bayArea,
    MAX(CASE WHEN (EXTRACT(Dayofweek FROM (SELECT PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING)))AT TIME ZONE "UTC"))=1 OR(EXTRACT(Dayofweek FROM (SELECT PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING)))AT TIME ZONE "UTC"))=7 THEN 1 ELSE 0 END) AS Weekend,
    MAX(CASE WHEN trafficSource.isTrueDirect = TRUE THEN 1 ELSE 0 END) AS TrafficSourceTrueDirect,
    MAX(CASE WHEN (EXTRACT(Quarter FROM (SELECT PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING)))AT TIME ZONE "UTC"))=1 THEN 1 ELSE 0 END )AS Q1,
    MAX(CASE WHEN (EXTRACT(Quarter FROM (SELECT PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING)))AT TIME ZONE "UTC"))=2 THEN 1 ELSE 0 END )AS Q2,
    MAX(CASE WHEN (EXTRACT(Quarter FROM (SELECT PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING)))AT TIME ZONE "UTC"))=3 THEN 1 ELSE 0 END )AS Q3,
    MAX(CASE WHEN (EXTRACT(Quarter FROM (SELECT PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING)))AT TIME ZONE "UTC"))=4 THEN 1 ELSE 0 END )AS Q4,
    MAX(EXTRACT(Month FROM (SELECT PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING)))AT TIME ZONE "UTC")) AS Month,
    MAX(EXTRACT(Week FROM (SELECT PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING)))AT TIME ZONE "UTC")) AS Week
    
FROM 
    `bigquery-public-data.google_analytics_sample.ga_sessions_*` ga
JOIN
    first_purchases fp
ON
    ga.fullVisitorId = fp.fullVisitorId
    AND PARSE_TIMESTAMP('%s', CAST(ga.visitStartTime AS string)) = fp.firstPurchaseSessionTime # Only getting data from first purchase
WHERE
    _TABLE_SUFFIX < '20170501' 
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
    AND geoNetwork.country = 'United States'
GROUP BY
    fp.fullVisitorId
),

first_purchases_hits AS (
# Get hit-level data from the session when the customer made their first purchase
# Hit data is stored in nested fields so we have to query it separately from the session-level data or else the session-level data aggregations would be incorrect
SELECT 
    fp.fullVisitorId,
    SUM(CASE WHEN hits.eCommerceAction.action_type = '6' THEN product.productQuantity ELSE 0 END) AS productQuantityPurchased, # Product quantity in the final transaction
    MAX(CASE WHEN product.v2ProductCategory LIKE "%Apparel%" AND hits.eCommerceAction.action_type = '6' THEN 1 ELSE 0 END) AS apparelPurchased, # dummy variable indicating if any apparel was purchased
    MAX(CASE WHEN product.v2ProductCategory LIKE "%Office%" AND hits.eCommerceAction.action_type = '6' THEN 1 ELSE 0 END) AS officePurchased,# dummy variable indicating if any office supplies were purchased
    MAX(CASE WHEN product.v2ProductCategory LIKE '%Drinkware%' AND hits.eCommerceAction.action_type = '6' THEN 1 ELSE 0 END) AS drinkwarePurchased,
    MAX(CASE WHEN product.v2ProductCategory = 'Lifestyle' AND hits.eCommerceAction.action_type = '6' THEN 1 ELSE 0 END) AS lifestylePurchased,
    MAX(CASE WHEN product.v2ProductCategory LIKE '%Bags%' AND hits.eCommerceAction.action_type = '6' THEN 1 ELSE 0 END) AS bagsPurchased,
    MAX(CASE WHEN product.v2ProductCategory LIKE '%Electronics%' AND hits.eCommerceAction.action_type = '6' THEN 1 ELSE 0 END) AS electronicsPurchased,# dummy variable indicating if any electronics were purchased
    MAX(CASE WHEN product.v2ProductCategory LIKE '%Android%' AND hits.eCommerceAction.action_type = '6' THEN 1 ELSE 0 END) AS BrandAndroidPurchased,
    MAX(CASE WHEN product.v2ProductCategory LIKE '%Google%' AND hits.eCommerceAction.action_type = '6' THEN 1 ELSE 0 END) AS BrandGooglePurchased,
    MAX(CASE WHEN product.v2ProductCategory LIKE '%YouTube%' AND hits.eCommerceAction.action_type = '6' THEN 1 ELSE 0 END) AS BrandYoutubePurchased,
    MAX(CASE WHEN hits.eCommerceAction.action_type = '4' THEN 1 ELSE 0 END) AS removedItemFromCart, # dummy variable indicating if they removed something from their cart during the session
    MAX(CASE WHEN hits.eCommerceAction.action_type = '3' THEN 1 ELSE 0 END) AS addedItemtoCart, 
    Max(CASE WHEN hits.social.hasSocialSourceReferral = 'Yes' THEN 1 ELSE 0 END) As HasSocialSourceReferral,
    MAX(product.productPrice/1000000) AS Maxproductprice
FROM 
    `bigquery-public-data.google_analytics_sample.ga_sessions_*` ga,
    UNNEST(hits) hits,
    UNNEST(hits.product) product
JOIN
    first_purchases fp
ON
    ga.fullVisitorId = fp.fullVisitorId
    AND PARSE_TIMESTAMP('%s', CAST(ga.visitStartTime AS string)) = fp.firstPurchaseSessionTime # Only getting data from first purchase
WHERE
    _TABLE_SUFFIX < '20170501' 
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
    AND geoNetwork.country = 'United States'
GROUP BY
    fp.fullVisitorId
),

# Get session level data that occured after the first purchase session but within 7 days of that session
# Looking to see if the user came back within a week and had any additional activity

first_week_sessions AS (
SELECT 
    fp.fullVisitorId,
    COALESCE(SUM(ga.totals.visits),0) AS firstWeekVisits, # Visits from first week of acquisition, excluding acquisition session
    COALESCE(SUM(ga.totals.transactions),0) AS firstWeekTransactions, # Transactions from first week of acquisition, excluding acquisition session
    COALESCE(ROUND(SUM(ga.totals.transactionRevenue/1000000),2),0) AS firstWeekRevenue, # Revenue from first week of acquisition, excluding acquisition session
    COALESCE(SUM(totals.bounces) / SUM(totals.visits),0) AS bounceRate,
    COALESCE(SUM(totals.transactions)/SUM(totals.visits),0) AS Frequency
FROM 
    first_purchases fp
LEFT JOIN
    `bigquery-public-data.google_analytics_sample.ga_sessions_*` ga
ON 
    fp.fullVisitorId = ga.fullVisitorId
    AND TIMESTAMP_DIFF(PARSE_TIMESTAMP('%s', CAST(ga.visitStartTime AS string)), fp.firstPurchaseSessionTime, HOUR) BETWEEN 1 AND 168 # Consider visits between 1 hour and 7 days after first purchase session
GROUP BY 
    fp.fullVisitorId
),

first_week_hits AS(
  SELECT 
    fp.fullVisitorId,
    COALESCE(MAX(CASE WHEN hits.eCommerceAction.action_type = '4' THEN 1 ELSE 0 END),0) AS removedItemFromCart_week, # dummy variable indicating if they removed something from their cart during the session
    COALESCE(MAX(CASE WHEN hits.eCommerceAction.action_type = '3' THEN 1 ELSE 0 END),0) AS addedItemtoCart_week, 
    COALESCE(MAX(CASE WHEN hits.social.hasSocialSourceReferral = 'Yes' THEN 1 ELSE 0 END),0) As HasSocialSourceReferral_week,
    COALESCE(MAX(CASE WHEN hits.promotionActionInfo IS NOT NULL THEN 1 ELSE 0 END),0) AS Promotion
FROM 
    `bigquery-public-data.google_analytics_sample.ga_sessions_*` ga

CROSS JOIN   UNNEST(hits) hits
Right JOIN
    first_purchases fp
ON 
    fp.fullVisitorId = ga.fullVisitorId
    AND TIMESTAMP_DIFF(PARSE_TIMESTAMP('%s', CAST(ga.visitStartTime AS string)), fp.firstPurchaseSessionTime, HOUR) BETWEEN 1 AND 168 # Consider visits between 1 hour and 7 days after first purchase session
GROUP BY 
    fp.fullVisitorId
),

future_purchases AS (
# For each newly acquired customer, look for any purchases that occurred in the next 8-90 days
SELECT
    fp.fullVisitorId,
    ROUND(SUM(ga.totals.transactionRevenue/1000000),2) AS futureRevenue # This is our model target
FROM 
    first_purchases fp
LEFT JOIN
    `bigquery-public-data.google_analytics_sample.ga_sessions_*` ga
ON 
    fp.fullVisitorId = ga.fullVisitorId
    AND TIMESTAMP_DIFF(PARSE_TIMESTAMP('%s', CAST(ga.visitStartTime AS string)), fp.firstPurchaseSessionTime, DAY) BETWEEN 8 AND 90 # Consider purchases between 8-90 days after acquisition
WHERE 
    ga.totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY 
    fp.fullVisitorId
),
#Create daytime variable in different timezone for customers' first purchase.
Daytime_All AS(
  SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="New York" 
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Los_Angeles"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="California"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Chicago"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Illinois"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Los_Angeles"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Washington"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Chicago"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Texas"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Michigan"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Massachusetts"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="District of Columbia"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Georgia"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Virginia"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Pennsylvania"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Denver"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Colorado"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="North Carolina"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Denver"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Arizona"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Florida"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="New Jersey"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Tennessee"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Ohio"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Los_Angeles"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Oregon"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Chicago"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Minnesota"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Chicago"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Nebraska"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Chicago"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Missouri"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Chicago"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Iowa"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="South Carolina"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Los_Angeles"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Nevada"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/Denver"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Utah"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Indiana"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 

UNION ALL
SELECT fullVisitorId, 
CASE WHEN (EXTRACT(HOUR FROM DATETIME(MIN(PARSE_TIMESTAMP("%s", CAST (GA.visitStartTime AS STRING))), "America/New_York"))) BETWEEN 7 AND 17 THEN 1 ELSE 0 END AS daytime,
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` GA
WHERE geoNetwork.region="Connecticut"
AND _TABLE_SUFFIX < '20170501' # The sample data ends in 08/2017 so we want to only look at acquisitions before 05/2017 to ensure they all have at least 90 days of subsequent data available
    AND totals.transactions > 0
    AND totals.transactionRevenue > 0
GROUP BY fullVisitorId 
)

# Join the list of newly acquired customers to their acquisition session-level data, acquisition hit-level data, first week session-level data, and future day 8-90 revenue
SELECT
    first_purchases.*,
    first_purchases_sessions.* EXCEPT(fullVisitorId),
    first_purchases_hits.* EXCEPT(fullVisitorId),
    first_week_sessions.* EXCEPT(fullVisitorId),
    COALESCE(future_purchases.futureRevenue,0) AS futureRevenue,
    Daytime_All.*EXCEPT(fullVisitorId),
    first_week_hits.* EXCEPT(fullVisitorId)
FROM 
    first_purchases
LEFT JOIN 
    first_purchases_sessions 
USING(fullVisitorId)
LEFT JOIN 
    first_purchases_hits 
USING(fullVisitorId)
LEFT JOIN 
    future_purchases 
USING(fullVisitorId)
LEFT JOIN
    first_week_sessions 
USING(fullVisitorId)
LEFT JOIN
Daytime_All
USING(fullVisitorId)
LEFT JOIN
first_week_hits
USING(fullVisitorId)
ORDER BY fullVisitorId
"""

# Store query results as Panada dataframe
df = pd.read_gbq(sql, project_id=project, dialect='standard', use_bqstorage_api=True)

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery_storage_v1beta1/reader.py:603: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


In [ ]:
# Create development and holdout datasets
# Dev data will be further split into training and testing data
# The holdout data will be used to get a final estimate of the model's performance
dev_data = df[df['firstPurchaseSessionTime'] < '2017-04-01 00:00:00 UTC']
holdout_data = df[df['firstPurchaseSessionTime'] >= '2017-04-01 00:00:00 UTC']

In [ ]:
dev_data.head()

,fullVisitorId,firstPurchaseSessionTime,revenue,visitNumber,hits,pageviews,timeOnSite,referralChannelGrouping,organicSearchChannelGrouping,directChannelGrouping,...,firstWeekTransactions,firstWeekRevenue,bounceRate,Frequency,futureRevenue,daytime,removedItemFromCart_week,addedItemtoCart_week,HasSocialSourceReferral_week,Promotion
1,000435324061339869,2016-10-21 01:33:41+00:00,44.79,2,14,11,627,1,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0
2,0007617910709180468,2016-12-12 16:23:52+00:00,18.99,3,15,13,246,0,1,0,...,0,0.0,0.0,0.0,0.0,NaN,0,0,0,0
4,0014253006455543633,2017-02-19 23:01:20+00:00,55.99,1,19,18,449,0,1,0,...,0,0.0,0.0,0.0,0.0,NaN,0,0,0,0
5,0015950283479889703,2017-02-26 22:35:50+00:00,1.50,4,19,18,419,0,1,0,...,0,0.0,0.0,0.0,0.0,NaN,0,0,0,0
6,0018386080117189534,2017-03-09 03:01:31+00:00,10.99,2,19,17,2212,0,1,0,...,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0


In [ ]:
holdout_data.head()

,fullVisitorId,firstPurchaseSessionTime,revenue,visitNumber,hits,pageviews,timeOnSite,referralChannelGrouping,organicSearchChannelGrouping,directChannelGrouping,...,firstWeekTransactions,firstWeekRevenue,bounceRate,Frequency,futureRevenue,daytime,removedItemFromCart_week,addedItemtoCart_week,HasSocialSourceReferral_week,Promotion
0,0000213131142648941,2017-04-28 22:41:58+00:00,33.59,1,14,13,272,0,0,1,...,0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
3,0013296981549010416,2017-04-22 01:23:13+00:00,16.99,1,26,21,651,0,1,0,...,0,0.0,0.0,0.0,0.0,NaN,0,0,0,0
17,0032285388804234573,2017-04-25 15:35:10+00:00,139.50,1,28,26,758,0,0,1,...,0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
19,0035322769016923863,2017-04-12 20:33:40+00:00,374.36,1,116,90,2516,1,0,0,...,0,0.0,0.0,0.0,0.0,1.0,0,0,0,0
30,0045204488424585972,2017-04-07 18:51:23+00:00,59.97,2,10,8,110,0,0,1,...,0,0.0,0.0,0.0,0.0,NaN,0,0,0,0


## Exploratory Data Analysis

In [ ]:
!pip install pycaret
#from pycaret.utils import enable_colab
#enable_colab()
from pycaret.regression import *

eda_data = dev_data.drop(['fullVisitorId', 'firstPurchaseSessionTime'], axis=1) # Exclude fullVisitorId and firstPurchaseSessionTime from EDA
eda_data = eda_data.reset_index(drop=True)

pc_eda = setup(data = eda_data, target = 'futureRevenue', session_id=123 , profile=True)

Based on the exploratory data analysis, we can see the following facts:


* We found that some of the dummy variables we created only got 0 results, so we removed those variables.
* Many numeric variables contain outliers. Since data outliers will mislead the machine learning  training process, resulting in less accurate models and poorer results, we decided to drop some outliers in the model training process.
* Almost 93% of futureRavenue values are 0. This will likely present a challenge for modeling.
* Only few variables that have high correlations with the futureRevenue.